<a href="https://colab.research.google.com/github/botelhorui/CloudPrime/blob/master/deploy_couchdb_to_flyio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Configurations
import os
os.environ['region']="otp"
os.environ['couchUser']="Ohr6yiu0"
os.environ['couchPwd']="soom2ei1DaeYohx3"


In [ ]:
# Delete once
!rm ./fly.toml
!rm ./local.ini
!rm ./Dockerfile

In [ ]:
# Installation
# You have to set up your account in here.
!curl -L https://fly.io/install.sh | sh
!/root/.fly/bin/flyctl auth signup

In [ ]:
# Generate local.ini
!echo -e "[couchdb]\nsingle_node=true\nmax_document_size = 50000000\n\n[chttpd]\nrequire_valid_user = true\nmax_http_request_size = 4294967296\n\n[chttpd_auth]\nrequire_valid_user = true\nauthentication_redirect = /_utils/session.html\n\n[httpd]\nWWW-Authenticate = Basic realm=\"couchdb\"\nenable_cors = true\n\n[cors]\norigins = app://obsidian.md,capacitor://localhost,http://localhost\ncredentials = true\nheaders = accept, authorization, content-type, origin, referer\nmethods = GET, PUT, POST, HEAD, DELETE\nmax_age = 3600" >> ./local.ini
!cat ./local.ini

In [ ]:
# Generate Dockerfile
!echo -e "FROM couchdb:latest\nADD ./local.ini /opt/couchdb/etc/local.ini" >> ./Dockerfile
!cat ./Dockerfile

In [ ]:
# Ignore sample_data directory from docker context since it's 50mb and slows the build down
!echo "sample_data" >> ./.dockerignore

In [ ]:
# Generate server
!/root/.fly/bin/flyctl launch  --auto-confirm --generate-name --detach --no-deploy --region ${region}
!/root/.fly/bin/fly volumes create --region ${region} couchdata --size 2 --yes

In [ ]:
# Modify fly.toml 
!cat ./fly.toml | grep -iv "\[env\]" | sed -e 's/8080/5984/g' > fly.tmp
!cat fly.tmp > ./fly.toml
!echo -e "\n[env]\n  COUCHDB_USER = \"${couchUser}\"\n\n[mounts]\n  source=\"couchdata\"\n  destination=\"/opt/couchdb/data\"" >> ./fly.toml

In [ ]:
# Show config
!cat ./fly.toml

In [ ]:
# Configure
!/root/.fly/bin/flyctl secrets set COUCHDB_PASSWORD=${couchPwd}

In [ ]:
# Deploy server
# Be sure to shutdown after the test.
!/root/.fly/bin/flyctl deploy --detach --remote-only
!/root/.fly/bin/flyctl status 

In [ ]:
import subprocess, json
result = subprocess.run(["/root/.fly/bin/flyctl","status","-j"], capture_output=True, text=True)
if result.returncode==0:
    hostname = json.loads(result.stdout)["Hostname"]
    os.environ['couchHost']="https://%s" % (hostname)
    print("Your couchDB server is https://%s/" % (hostname))
else:
    print("Something occured.")
